In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import yfinance as yf

In [10]:
empresas = pd.read_csv('empresas_v2.csv')
infos = pd.read_csv('infos_empresas.csv')
cotacoes = pd.read_csv('cotacoes.csv')
dividendos = pd.read_csv('dividendos.csv')
indicadores = pd.read_csv('indicadores.csv')

empresas = empresas.drop(empresas.columns[0], axis=1)
infos = infos.drop(infos.columns[0], axis=1)
cotacoes = cotacoes.drop(cotacoes.columns[0], axis=1)
dividendos = dividendos.drop(dividendos.columns[0], axis=1)
indicadores = indicadores.drop(indicadores.columns[0], axis=1)

In [12]:
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa,dividendo
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
1,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778
2,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987,0.487119
3,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195,1.288606
4,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075,1.450221
...,...,...,...,...,...,...,...,...,...,...
251,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563,0.236894
252,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000,0.465888
253,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602,0.549765
254,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000,0.432831


In [3]:
cotacao_list = []
for ticker in empresas.empresa:
    ticker = ticker + '.SA'

    acao = yf.Ticker(ticker)

    try:
        cotação_atualizada = acao.history(period="1d")["Close"].iloc[-1]
    except:
        cotação_atualizada = 0

    cotacao_list.append(cotação_atualizada)

BRSR5.SA: No price data found, symbol may be delisted (period=1d)


In [4]:
empresas['cotacao'] = cotacao_list

In [5]:
multiplos = {
    'energia': {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'saneamento' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'banco' : {'vpa':1.4,
                'roe': 0.15,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.6},
    'seguro' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 12,
                'dy': 8,
                'estatal_mult': 0.7},
    'telecom' : {'vpa':1.0,
                'roe': 0.10,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.7},
    'combustivel' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7}
}

In [6]:
df_list_indicadores_val = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    cotacoes_empresa = cotacoes[cotacoes['ticker']==ticker]
    cotacoes_empresa_ano = cotacoes_empresa[cotacoes_empresa['year']==ano]

    try:
        pl_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['lpa']
        pl_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['lpa']
        pl_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['lpa']
    except:
        pl_max = 0
        pl_min = 0
        pl_clo = 0

    try:
        pvp_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['vpa']
        pvp_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['vpa']
        pvp_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['vpa']
    except:
        pvp_max = 0
        pvp_min = 0
        pvp_clo = 0

    try:
        dy_max = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['max'].values[0]
        dy_min = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['min'].values[0]
        dy_clo = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['close'].values[0]
    except:
        dy_max = 0
        dy_min = 0
        dy_clo = 0

    dicionario = {
        'ticker': ticker,
        'year': ano,

        'pl':pl_clo,
        'pl_max': pl_max,
        'pl_min': pl_min,

        'pvp':pvp_clo,
        'pvp_max': pvp_max,
        'pvp_min': pvp_min,

        'dy':dy_clo,
        'dy_max': dy_max,
        'dy_min': dy_min
    }

    indicadores_values = pd.DataFrame(dicionario, index=[0])
    df_list_indicadores_val.append(indicadores_values)

C:\Users\eduar\AppData\Local\Temp\ipykernel_16760\1100314948.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_16760\1100314948.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_16760\1100314948.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_16760\1100314948.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  pl_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['lpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_16760\1100314948.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  pl_min = cotacoes_empresa_ano['min'].values[0]/in

In [7]:
faixas_indicadores = pd.concat(df_list_indicadores_val)

In [8]:
df_list_precos = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    estatal = empresas.iloc[i]['estatal']
    equivalencia_unit = empresas.iloc[i]['equivalencia_unit']
    cotacao = empresas.iloc[i]['cotacao']

    dividendos_por_acao_total = indicadores[indicadores['ticker']==ticker]['dividendo'].mean()
    dividendos_por_acao_3_dados = indicadores[indicadores['ticker']==ticker]['dividendo'].iloc[-3:].mean()
    dividendos_por_acao_5_dados = indicadores[indicadores['ticker']==ticker]['dividendo'].iloc[-5:].mean()

    roe = indicadores[indicadores['ticker']==ticker]['roe'].iloc[-1]

    ultimo_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].iloc[-1]
    media_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].mean()
    media_ultimos_3_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].iloc[-3:].mean()

    media_pl_min = faixas_indicadores[faixas_indicadores['ticker']==ticker]['pl_min'].iloc[-3:].mean()

    ultimo_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].iloc[-1]
    media_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].mean()
    media_ultimos_3_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].iloc[-3:].mean()

    media_pvp_min = faixas_indicadores[faixas_indicadores['ticker']==ticker]['pvp_min'].iloc[-3:].mean()

    indice_pvp = multiplos[setor]['vpa']
    if estatal:
        indice_pvp = indice_pvp * multiplos[setor]['estatal_mult']

    indice_pl = multiplos[setor]['pl']
    if estatal:
        indice_pl = indice_pl * multiplos[setor]['estatal_mult']

    indice_div = multiplos[setor]['dy']
    indice_div = indice_div/100

    preco_teto_3_8 = dividendos_por_acao_3_dados/indice_div
    if media_ultimos_3_vpa > preco_teto_3_8:
        preco_teto_3_8_ajustato = (media_ultimos_3_vpa - preco_teto_3_8)/2 + preco_teto_3_8
    else:
        preco_teto_3_8_ajustato = preco_teto_3_8

    preco_teto_vpa = media_ultimos_3_vpa * indice_pvp * equivalencia_unit
    if cotacao > preco_teto_vpa*1.5 or preco_teto_vpa > cotacao * 0.5:
        preco_teto_vpa_ajustado = media_pvp_min * cotacao
    else:
        preco_teto_vpa_ajustado = preco_teto_vpa

    preco_teto_lpa = media_ultimos_3_lpa * indice_pl * equivalencia_unit
    if cotacao > preco_teto_lpa*1.5 or preco_teto_lpa > cotacao * 0.5:
        preco_teto_lpa_ajustado = media_pl_min * cotacao
    else:
        preco_teto_lpa_ajustado = preco_teto_lpa


    dicionario={
        'ticker': ticker,
        'estatal': estatal,
        'equivalencia_unit': equivalencia_unit,
        'div_pa': dividendos_por_acao_total,
        'div3_pa': dividendos_por_acao_3_dados,
        'lpa': ultimo_lpa,
        'l3pa': media_ultimos_3_lpa,
        'vpa': ultimo_vpa,
        'v3pa': media_ultimos_3_vpa,
        'indice_pl': indice_pl,
        'indice_pvp': indice_pvp,
        'indice_div': indice_div,
        'preco_3_8': preco_teto_3_8,
        'preco_3_8_ajustado': preco_teto_3_8_ajustato,
        'preco_teto_vpa': preco_teto_vpa,
        'preco_teto_vpa_ajustado': preco_teto_vpa_ajustado,
        'preco_teto_lpa': preco_teto_lpa,
        'preco_teto_lpa_ajustado': preco_teto_lpa_ajustado,
    }

    df=pd.DataFrame(dicionario,index=[0])
    
    df_list_precos.append(df) 


NameError: name 'setor' is not defined